<a href="https://colab.research.google.com/github/dakilaledesma/VIGOR2/blob/main/notebooks/%5BNI%5D_Kinematics_to_Music_Protomodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
! curl -L https://herbarium.ucht.cloud/s/AoZFbaKWofEfHiF/download --output motions.zip
! curl -L https://herbarium.ucht.cloud/s/Cn55KNbEBqtrDCg/download --output mp3.zip
! curl -L https://herbarium.ucht.cloud/s/MayD6rAGWoKmGw8/download --output keypoints3d.zip
! unzip -q motions.zip
! unzip -q keypoints3d.zip
! unzip -q mp3.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  305M  100  305M    0     0  47.0M      0  0:00:06  0:00:06 --:--:-- 60.5M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 90.6M  100 90.6M    0     0  32.0M      0  0:00:02  0:00:02 --:--:-- 32.0M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  834M  100  834M    0     0  49.5M      0  0:00:16  0:00:16 --:--:-- 58.0M
replace motions/gWA_sBM_cAll_d25_mWA3_ch07.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
replace keypoints3d/gJB_sBM_cAll_d08_mJB4_ch04.pkl? [y]es, [n]o, [A]ll, [N]one, [r]ename: replace mp3/mKR5.mp3? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [21]:
audio_dict = {}
motion_dict = {}

In [23]:
# Thanks to https://stackoverflow.com/questions/69387104/how-to-convert-wav-audio-file-from-mel-spectrogram
import librosa
import os
from glob import glob

for aud in glob("mp3/*.mp3"):
  my_audio_as_np_array, my_sample_rate = librosa.load(aud)

  spec = librosa.feature.melspectrogram(y=my_audio_as_np_array,
                                          sr=my_sample_rate, 
                                              n_fft=2048, 
                                              hop_length=512, 
                                              win_length=None, 
                                              window='hann', 
                                              center=True, 
                                              pad_mode='reflect', 
                                              power=2.0,
                                      n_mels=128)
  spec = spec.T[:1272].T # Minimum is (128, 1273)
  bn = os.path.basename(aud).replace(".mp3", '')
  audio_dict[bn] = spec

In [30]:
import pickle
import numpy as np
import matplotlib.pyplot as plt

for fn in glob("keypoints3d/*.pkl"):
  with open(fn, "rb") as motion:
    m = pickle.load(motion)
    bn = os.path.basename(fn).replace(".pkl", '')
    motion_dict[bn] = m["keypoints3d"][:426] # Minimum is (426, 17, 3), axis=0 is all different

In [31]:
print(motion_dict.keys())

dict_keys(['gLH_sBM_cAll_d18_mLH5_ch06', 'gBR_sBM_cAll_d04_mBR0_ch03', 'gKR_sFM_cAll_d29_mKR5_ch13', 'gMH_sBM_cAll_d24_mMH5_ch02', 'gWA_sBM_cAll_d26_mWA5_ch04', 'gJB_sFM_cAll_d08_mJB0_ch08', 'gLO_sBM_cAll_d13_mLO1_ch03', 'gKR_sBM_cAll_d28_mKR3_ch09', 'gHO_sBM_cAll_d19_mHO3_ch02', 'gJS_sBM_cAll_d03_mJS5_ch08', 'gKR_sBM_cAll_d29_mKR0_ch07', 'gKR_sBM_cAll_d30_mKR4_ch10', 'gMH_sBM_cAll_d24_mMH5_ch07', 'gKR_sBM_cAll_d30_mKR3_ch10', 'gLH_sBM_cAll_d18_mLH4_ch05', 'gBR_sBM_cAll_d05_mBR1_ch02', 'gBR_sBM_cAll_d06_mBR4_ch09', 'gLH_sBM_cAll_d18_mLH4_ch01', 'gWA_sBM_cAll_d26_mWA0_ch06', 'gHO_sBM_cAll_d20_mHO5_ch10', 'gKR_sFM_cAll_d28_mKR3_ch07', 'gLH_sBM_cAll_d17_mLH5_ch06', 'gKR_sBM_cAll_d30_mKR3_ch06', 'gMH_sBM_cAll_d22_mMH0_ch08', 'gLH_sFM_cAll_d16_mLH2_ch03', 'gWA_sFM_cAll_d26_mWA2_ch10', 'gLH_sBM_cAll_d18_mLH5_ch10', 'gMH_sBM_cAll_d22_mMH1_ch03', 'gKR_sBM_cAll_d29_mKR0_ch04', 'gLO_sBM_cAll_d14_mLO4_ch03', 'gKR_sBM_cAll_d29_mKR5_ch03', 'gLH_sFM_cAll_d16_mLH0_ch01', 'gLO_sBM_cAll_d15_mLO2_ch04',

In [32]:
print(audio_dict.keys())

dict_keys(['mJB1', 'mPO4', 'mWA2', 'mWA1', 'mBR4', 'mJB4', 'mLO1', 'mWA0', 'mLH0', 'mBR0', 'mHO3', 'mLH1', 'mMH0', 'mKR0', 'mHO2', 'mLO3', 'mMH5', 'mKR3', 'mBR1', 'mKR1', 'mKR2', 'mJS4', 'mLO4', 'mLH5', 'mJB3', 'mJB0', 'mLO2', 'mKR5', 'mJS1', 'mWA4', 'mPO1', 'mLH2', 'mBR3', 'mMH1', 'mPO2', 'mMH3', 'mLH4', 'mLH3', 'mMH4', 'mBR5', 'mPO0', 'mLO0', 'mHO0', 'mJB5', 'mHO4', 'mBR2', 'mKR4', 'mHO5', 'mJB2', 'mPO3', 'mJS5', 'mJS0', 'mLO5', 'mWA5', 'mJS2', 'mPO5', 'mHO1', 'mWA3', 'mMH2', 'mJS3'])


In [33]:
X = []
y = []
for ak in audio_dict.keys():
  mks = [k for k in motion_dict.keys() if ak in k]
  for mk in mks:
    X.append(motion_dict[mk])
    y.append(audio_dict[ak])

In [34]:
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

(1408, 426, 17, 3) (1408, 128, 1272)


In [39]:
import tensorflow.keras as keras
from tensorflow.keras import layers

input_motion = keras.Input(shape=(426, 17, 3))

x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(input_motion)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

z = layers.Flatten()(encoded)
x = layers.Reshape((64, 1), input_shape=(64,))(z)
x = layers.Conv1D(256, (3), activation='relu', padding='same')(x)
x = layers.Conv1D(256, (3), activation='relu', padding='same')(x)
x = layers.UpSampling1D((2))(x)
decoded = layers.Conv1D(1272, (1), activation='linear', padding='same')(x)

autoencoder = keras.Model(input_motion, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

In [40]:
autoencoder.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 426, 17, 3)]      0         
                                                                 
 conv2d_114 (Conv2D)         (None, 426, 17, 128)      3584      
                                                                 
 conv2d_115 (Conv2D)         (None, 426, 17, 128)      147584    
                                                                 
 max_pooling2d_57 (MaxPoolin  (None, 213, 9, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_116 (Conv2D)         (None, 213, 9, 64)        73792     
                                                                 
 conv2d_117 (Conv2D)         (None, 213, 9, 64)        36928     
                                                           

In [41]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [ ]:
autoencoder.fit(X, y, 64, epochs=1)

 7/22 [========>.....................] - ETA: 3:19 - loss: nan

In [ ]:
res = librosa.feature.inverse.mel_to_audio(spec, 
                                           sr=my_sample_rate, 
                                           n_fft=2048, 
                                           hop_length=512, 
                                           win_length=None, 
                                           window='hann', 
                                           center=True, 
                                           pad_mode='reflect', 
                                           power=2.0, 
                                           n_iter=32)

import soundfile as sf
sf.write("test1.wav", res, sashi_sr)